<a href="https://colab.research.google.com/github/StyrbjornKall/TRIDENT/blob/master/tutorials/Inference_tutorial_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inference
This script showcases the different models available in TRIDENT and how to use them efficiently.

This is the exact same tutorial as the jupyter notebook available under `tutorials` but since this one runs on google colab some additional code need to run for it to work.

Note: For large files it is recommended to switch the Runtime to GPU (select *GPU* under the *Change Runtime type* in the dropdown menu *Runtime*).  

## Install dependencies

In [ ]:
!pip install transformers
!pip install torch
!pip install rdkit==2022.03.5

## Mount personal google drive
The paths stated below should not have to be changed for functional code. The script will automatically make a new folder called `TRIDENT` in you google drive.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd gdrive/My Drive/
%ls

In [4]:
!git clone https://github.com/StyrbjornKall/TRIDENT

Cloning into 'TRIDENT'...
remote: Enumerating objects: 1506, done.
remote: Counting objects: 100% (256/256), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 1506 (delta 139), reused 192 (delta 93), pack-reused 1250
Receiving objects: 100% (1506/1506), 1.42 GiB | 18.51 MiB/s, done.
Resolving deltas: 100% (874/874), done.
Updating files: 100% (157/157), done.


In [5]:
import os
os.chdir('/content/gdrive/My Drive/TRIDENT/tutorials/')

## Run tutorial

Now we are ready to run the script. This follows the exact same layout as the jupyter notebook tutorial available under `tutorials`.

In [6]:
import torch
import pandas as pd
import numpy as np
from inference_utils.TRIDENT_for_inference import TRIDENT_for_inference

Specify the model version and load the model

In [7]:
MODEL_TYPE = 'EC50'
SPECIES_GROUP = 'fish'
MODEL_VERSION = f'{MODEL_TYPE}_{SPECIES_GROUP}'

In [8]:
trident = TRIDENT_for_inference(model_version=MODEL_VERSION)
trident.load_fine_tuned_model()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/334M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/149k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/101k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/384k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

Loading DNN... 

Model loaded.



Load the SMILES you wish to predict

In [10]:
data = pd.read_excel('../data/tutorials/Inference_example_2.xlsx')
data

,SMILES,cmpdname
0,CC(=O)Oc1ccccc1C(O)=O,Aspirin
1,[Cr],Chromium
2,[H+].[Cl-].CNCCC(Oc1ccc(cc1)C(F)(F)F)c2ccccc2,Fluoxetine hydrochloride
3,Clc1ccc(cc1)C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl,Clofenotane
4,[Cu],Copper
...,...,...
995,[Pb++].[O-]c1c(cc(c([O-])c1[N+]([O-])=O)[N+]([...,Lead styphnate
996,CC(C)(C)C(O)(CCc1ccc(Cl)cc1)Cn2cncn2,Tebuconazole
997,[Na+].[Na+].[Na+].[Na+].OCCN(CCO)c1nc(Nc2ccc(c...,OpticalBrightenerBbu220
998,CNC.OC(=O)COc1ccc(Cl)cc1Cl,"2,4-D dimethylamine salt"


Specify the endpoint and effect you wish to predict and make the prediction

In [11]:
PREDICTION_ENDPOINT = 'EC50'
PREDICTION_EFFECT = 'MOR'
EXPOSURE_DURATION = 96
SMILES_COLUMN_NAME = 'SMILES'

In [12]:
results = trident.predict_toxicity(SMILES = data[SMILES_COLUMN_NAME].iloc[0:10].tolist(), exposure_duration=EXPOSURE_DURATION, endpoint=PREDICTION_ENDPOINT, effect=PREDICTION_EFFECT, return_cls_embeddings=True)
results

Did not return onehotencoding for Endpoint. Why? You specified only one Endpoint or you specified NOEC and EC10 which are coded to be the same endpoint.
Did not return onehotencoding for Effect. Why? You specified only one Effect.
Will use input 0 to network due to no Onehotencodings being present.


100%|██████████| 2/2 [00:02<00:00,  1.03s/it]


,SMILES,exposure_duration log10(h),endpoint,effect,SMILES_Canonical_RDKit,OneHotEnc_concatenated,predictions log10(mg/L),predictions (mg/L),CLS_embeddings
0,CC(=O)Oc1ccccc1C(O)=O,1.982271,EC50,MOR,CC(=O)Oc1ccccc1C(=O)O,[0.0],1.745711,55.681553,"[0.6759762167930603, -0.29787713289260864, 1.7..."
1,[Cr],1.982271,EC50,MOR,[Cr],[0.0],1.779521,60.189583,"[0.5223562121391296, -0.5395256876945496, 2.18..."
2,[H+].[Cl-].CNCCC(Oc1ccc(cc1)C(F)(F)F)c2ccccc2,1.982271,EC50,MOR,CNCCC(Oc1ccc(C(F)(F)F)cc1)c1ccccc1.[Cl-].[H+],[0.0],-0.180053,0.660613,"[-0.8463513255119324, -0.3668559193611145, 0.4..."
3,Clc1ccc(cc1)C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl,1.982271,EC50,MOR,Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1,[0.0],-1.832513,0.014706,"[-1.0411754846572876, -0.12303479760885239, -0..."
4,[Cu],1.982271,EC50,MOR,[Cu],[0.0],-0.401859,0.396406,"[-1.542029857635498, 0.9921452403068542, 0.299..."
5,CCNc1nc(Cl)nc(NC(C)C)n1,1.982271,EC50,MOR,CCNc1nc(Cl)nc(NC(C)C)n1,[0.0],1.273837,18.786098,"[0.5072570443153381, -1.1547664403915405, 1.78..."
6,CN(C)C1=NC(=O)N(C2CCCCC2)C(=O)N1C,1.982271,EC50,MOR,CN(C)c1nc(=O)n(C2CCCCC2)c(=O)n1C,[0.0],2.440936,276.016998,"[0.7960097193717957, 0.48650991916656494, 2.32..."
7,CC(Br)(CO)[N+]([O-])=O,1.982271,EC50,MOR,CC(Br)(CO)[N+](=O)[O-],[0.0],0.585505,3.850391,"[-0.8459401726722717, -0.3140878677368164, 1.4..."
8,c1ccc2c(c1)c3cccc4cccc2c34,1.982271,EC50,MOR,c1ccc2c(c1)-c1cccc3cccc-2c13,[0.0],-1.450543,0.035437,"[-1.3782415390014648, 0.447883665561676, -0.27..."
9,[Cl-].[Cl-].[Zn++],1.982271,EC50,MOR,[Cl-].[Cl-].[Zn+2],[0.0],0.278885,1.900576,"[-1.3507226705551147, 0.28230974078178406, 0.3..."


If you run into truble using this model use the __help__ function

In [13]:
trident.__help__()


        This is a python class used to load and use the fine-tuned deep-learning model `TRIDENT` for environmental toxicity predictions in fish algae and aquatic invertebrates.
        The models have been trained on a large corpus of SMILES (chemical representations) on data collected from various sources.

        Currently there are nine models available for use. The models are divided by toxicity endpoint and by species group. The models are the following:
        **Fish**
        - `EC50_fish` The EC50 model is trained on EC50 mortality (MOR) data and is thus suitable for the prediction of said endpoints.
        - `EC10_fish` The EC10 model is trained on EC10/NOEC data with various effects (mortality, intoxication, development, reproduction, morphology, growth and population) ab. (MOR, ITX, DVP, REP, MPH, GRO, POP)
        - `EC50EC10_fish` The EC50EC10 model is trained on EC50, EC10 and NOEC data with various effects (mortality, intoxication, development, reproduction, morpholo

## Upload your list of SMILES

For simplicity a file can be uploaded with the SMILES you wish to predict directly

In [14]:
from google.colab import files
import io

In [40]:
uploaded_file = files.upload()

Saving smiles.txt to smiles (3).txt


In [41]:
# excel file with one column containing SMILES
#data = pd.read_excel(io.BytesIO(uploaded_file[list(uploaded_file.keys())[0]]), header=None, names=['SMILES'])

# .csv file with one column containing SMILES
#data = pd.read_csv(io.BytesIO(uploaded_file[list(uploaded_file.keys())[0]]), header=None, names=['SMILES'])

# .txt file with one SMILES per line
data = pd.read_csv(io.BytesIO(uploaded_file[list(uploaded_file.keys())[0]]), sep='\rn', header=None, names=['SMILES'])

<ipython-input-41-4e1a3720811a>:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(io.BytesIO(uploaded_file[list(uploaded_file.keys())[0]]), sep='\rn', header=None, names=['SMILES'])


In [42]:
data

,SMILES
0,CC(C)CCON=O
1,CCC12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@H]34)[C@@H...
2,CCN(Cc1cccc(c1)[S](O)(=O)=O)c2ccc(cc2)[C+](c3c...
3,[Ni++].OC(=O)CN(CCN(CC(O)=O)CC([O-])=O)CC([O-])=O
4,[B+3].[H+].[H+].[F-].[F-].[F-].CC([O-])=O.CC([...


In [29]:
PREDICTION_ENDPOINT = 'EC50'
PREDICTION_EFFECT = 'MOR'
EXPOSURE_DURATION = 96
SMILES_COLUMN_NAME = 'SMILES'

In [43]:
results = trident.predict_toxicity(SMILES = data[SMILES_COLUMN_NAME].tolist(),
                                   exposure_duration=EXPOSURE_DURATION,
                                   endpoint=PREDICTION_ENDPOINT,
                                   effect=PREDICTION_EFFECT,
                                   return_cls_embeddings=True)
results

Did not return onehotencoding for Endpoint. Why? You specified only one Endpoint or you specified NOEC and EC10 which are coded to be the same endpoint.
Did not return onehotencoding for Effect. Why? You specified only one Effect.
Will use input 0 to network due to no Onehotencodings being present.


100%|██████████| 1/1 [00:00<00:00, 45.86it/s]


,SMILES,exposure_duration log10(h),endpoint,effect,SMILES_Canonical_RDKit,OneHotEnc_concatenated,predictions log10(mg/L),predictions (mg/L),CLS_embeddings
0,CC(C)CCON=O,1.982271,EC50,MOR,CC(C)CCON=O,[0.0],1.363915,23.116121,"[0.6714611649513245, -1.382544755935669, 1.347..."
1,CCC12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@H]34)[C@@H...,1.982271,EC50,MOR,CCC12CC[C@@H]3[C@H]4CCC(=O)C=C4CC[C@H]3[C@@H]1...,[0.0],0.575924,3.766376,"[0.8218030333518982, 0.43655648827552795, 0.95..."
2,CCN(Cc1cccc(c1)[S](O)(=O)=O)c2ccc(cc2)[C+](c3c...,1.982271,EC50,MOR,CCN(Cc1cccc(S(=O)(=O)[O-])c1)c1ccc([C+](c2ccc(...,[0.0],2.299701,199.388901,"[0.9449657797813416, 0.9571866989135742, 1.843..."
3,[Ni++].OC(=O)CN(CCN(CC(O)=O)CC([O-])=O)CC([O-])=O,1.982271,EC50,MOR,O=C([O-])CN(CCN(CC(=O)[O-])CC(=O)O)CC(=O)O.[Ni+2],[0.0],2.142261,138.759033,"[1.0897550582885742, 0.37517309188842773, 2.25..."
4,[B+3].[H+].[H+].[F-].[F-].[F-].CC([O-])=O.CC([...,1.982271,EC50,MOR,CC(=O)[O-].CC(=O)[O-].[B+3].[F-].[F-].[F-].[H+...,[0.0],2.011792,102.752449,"[1.293481469154358, -0.16804705560207367, 1.64..."


# Check the predictions compared to our training sets

In [ ]:
!pip install umap-learn
!pip install pacmap

In [37]:
from inference_utils.plots_for_space import PlotPCA_CLSProjection, PlotPaCMAP_CLSProjection, PlotUMAP_CLSProjection
from inference_utils.pytorch_data_utils import check_closest_chemical, check_training_data

Check if chemicals are present in training data. They may be present as either an:

- 'endpoint_match' i.e. the chemical was used for training this model for this species and endpoint.
- 'effect_match' i.e. the chemical was used for training this model for this species, endpoint and effect.

In [44]:
results = check_training_data(results, model_type=MODEL_TYPE, species_group=SPECIES_GROUP, endpoint=PREDICTION_ENDPOINT, effect=PREDICTION_EFFECT)
results

,SMILES,exposure_duration log10(h),endpoint,effect,SMILES_Canonical_RDKit,OneHotEnc_concatenated,predictions log10(mg/L),predictions (mg/L),CLS_embeddings,endpoint match,effect match
0,CC(C)CCON=O,1.982271,EC50,MOR,CC(C)CCON=O,[0.0],1.363915,23.116121,"[0.6714611649513245, -1.382544755935669, 1.347...",0,0
1,CCC12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@H]34)[C@@H...,1.982271,EC50,MOR,CCC12CC[C@@H]3[C@H]4CCC(=O)C=C4CC[C@H]3[C@@H]1...,[0.0],0.575924,3.766376,"[0.8218030333518982, 0.43655648827552795, 0.95...",0,0
2,CCN(Cc1cccc(c1)[S](O)(=O)=O)c2ccc(cc2)[C+](c3c...,1.982271,EC50,MOR,CCN(Cc1cccc(S(=O)(=O)[O-])c1)c1ccc([C+](c2ccc(...,[0.0],2.299701,199.388901,"[0.9449657797813416, 0.9571866989135742, 1.843...",0,0
3,[Ni++].OC(=O)CN(CCN(CC(O)=O)CC([O-])=O)CC([O-])=O,1.982271,EC50,MOR,O=C([O-])CN(CCN(CC(=O)[O-])CC(=O)O)CC(=O)O.[Ni+2],[0.0],2.142261,138.759033,"[1.0897550582885742, 0.37517309188842773, 2.25...",0,0
4,[B+3].[H+].[H+].[F-].[F-].[F-].CC([O-])=O.CC([...,1.982271,EC50,MOR,CC(=O)[O-].CC(=O)[O-].[B+3].[F-].[F-].[F-].[H+...,[0.0],2.011792,102.752449,"[1.293481469154358, -0.16804705560207367, 1.64...",0,0


Next we check if which chemical is closest to the predicted chemicals by evaluating the CLS-embeddings against the training set's CLS-embedding by means of their cosine-similarity:

- cosine-similarity=1 --> Identical structures
- cosine-similarity=-1 --> completely oposite in terms of toxicity

In [45]:
results = check_closest_chemical(results=results, MODELTYPE=MODEL_TYPE, PREDICTION_SPECIES=SPECIES_GROUP, PREDICTION_ENDPOINT=PREDICTION_ENDPOINT, PREDICTION_EFFECT=PREDICTION_EFFECT)
results

,SMILES,exposure_duration log10(h),endpoint,effect,SMILES_Canonical_RDKit,OneHotEnc_concatenated,predictions log10(mg/L),predictions (mg/L),CLS_embeddings,endpoint match,effect match,most similar chemical,max cosine similarity,mean cosine similarity
0,CC(C)CCON=O,1.982271,EC50,MOR,CC(C)CCON=O,[0.0],1.363915,23.116121,"[0.6714611649513245, -1.382544755935669, 1.347...",0,0,CC(C)C=O,0.914274,0.408005
1,CCC12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@H]34)[C@@H...,1.982271,EC50,MOR,CCC12CC[C@@H]3[C@H]4CCC(=O)C=C4CC[C@H]3[C@@H]1...,[0.0],0.575924,3.766376,"[0.8218030333518982, 0.43655648827552795, 0.95...",0,0,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C...,0.706685,0.263146
2,CCN(Cc1cccc(c1)[S](O)(=O)=O)c2ccc(cc2)[C+](c3c...,1.982271,EC50,MOR,CCN(Cc1cccc(S(=O)(=O)[O-])c1)c1ccc([C+](c2ccc(...,[0.0],2.299701,199.388901,"[0.9449657797813416, 0.9571866989135742, 1.843...",0,0,CCN(CC)c1nc(Nc2ccc(/C=C/c3ccc(Nc4nc(Nc5cc(S(=O...,0.968660,0.284108
3,[Ni++].OC(=O)CN(CCN(CC(O)=O)CC([O-])=O)CC([O-])=O,1.982271,EC50,MOR,O=C([O-])CN(CCN(CC(=O)[O-])CC(=O)O)CC(=O)O.[Ni+2],[0.0],2.142261,138.759033,"[1.0897550582885742, 0.37517309188842773, 2.25...",0,0,O=C(O)CN(CCN(CC(=O)O)CC(=O)O)CC(=O)O,0.965087,0.343525
4,[B+3].[H+].[H+].[F-].[F-].[F-].CC([O-])=O.CC([...,1.982271,EC50,MOR,CC(=O)[O-].CC(=O)[O-].[B+3].[F-].[F-].[F-].[H+...,[0.0],2.011792,102.752449,"[1.293481469154358, -0.16804705560207367, 1.64...",0,0,O=S(=O)([O-])[O-].O=S(=O)([O-])[O-].[Fe+2].[NH...,0.949373,0.360724


# Plot projections for all chemicals
Finally we can plot the chemical space built during training of the Transformer module in the model. The space is built by the CLS-embedidngs present in the training set of the model but can be used to project new chemicals onto. The space prepared in this example uses show_all_predictions=True which plots additional SMILES, not included in the training data to add interpretability. We also use inference_df=results to plot the predicted SMILES from above into the space, however this can be set to None if not desired.

The plot can be saved as interactive HTML by fig.write_html(figurename.html')

Note that in the hover text of each point, the L1Error from our 10x10-fold cross-validation is included from when the used model was evaluated on that chemical.

## PCA - CLS projection

In [46]:
PlotPCA_CLSProjection(model_type=MODEL_TYPE, endpoint=PREDICTION_ENDPOINT, effect=PREDICTION_EFFECT, species_group=SPECIES_GROUP, show_all_predictions=True, inference_df=results)

## UMAP - CLS projection

In [47]:
PlotUMAP_CLSProjection(model_type=MODEL_TYPE, endpoint=PREDICTION_ENDPOINT, effect=PREDICTION_EFFECT, species_group=SPECIES_GROUP, show_all_predictions=True, inference_df=results, n_neighbors=10, min_dist=0.1)
